In [1]:
import os,os.path
import csv
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize

#  Create documents

In [19]:
# creating a .tsv document for each review
with open('Airbnb_Texas_Rentals.tsv', 'r', encoding = "utf8") as all_rev:
    csv_reader = csv.reader(all_rev)
    ind = -1
    for row in all_rev:
        ind += 1
        if ind == 0:
            continue
        # skipping the empty lines
        elif row == '\n':
            ind -= 1
            # we manipulate 'ind' in order to have files named doc_i where i = 1,2,3,..
        else:
            # we store the documents inside a new folder
            new_title = r'C:\Users\Daniele\fileAIR\doc_' + str(ind-1) + '.tsv'
            csv_writer = csv.writer(open(new_title, 'w', encoding = "utf8"), delimiter ='\t')
            # we skip the first column which is 
            row1 = row.split("\t")[1:]
            csv_writer.writerow(row1)

KeyboardInterrupt: 

# Search Engine

In [2]:
# function to preprocess the files
def preprocess(something):
    something = something.lower()
    # removing '\n'
    something = something.replace('\\n', ' ')
    # removing punctuation
    tokenizer = regexp_tokenize(something, "[\w'\$]+")
    # filter the non stopwords
    filtered = [w for w in tokenizer if not w in stopwords.words('english')]
    ps = PorterStemmer()
    # removing the stem
    filtered = [ps.stem(word) for word in filtered]
    return filtered

In [3]:
len_file = len([x for x in os.scandir(r'C:\Users\Daniele\fileAIR')])
print (len_file)

265


In [4]:
vocabulary_set = set()
docs_list = []

In [5]:
for i in range(0, len_file):
    with open(r'C:\Users\Daniele\fileAIR\doc_' + str(i) + '.tsv', 'r', encoding = 'utf8') as csvfile:
        file1 = csv.reader(csvfile, delimiter = '\t')
        columns = [i for i in file1]
        description = columns[0][4]
        description = preprocess(description)
        title = columns[0][7]
        title = preprocess(title)
        tit_desc = title + description
        vocabulary_set.update(tit_desc)
        docs_list.append(set(tit_desc))

In [6]:
vocabulary = {k:v for v, k in enumerate(vocabulary_set)}
#vocabulary

In [8]:
# save vocabulary to the file
voc_file = open(r'C:\Users\Daniele\vocabulary.txt', 'w', encoding = "utf8")
for term in vocabulary:
    voc_file.write('{0}\t{1}\n'.format(term, vocabulary[term]))

In [9]:
# we use defaultdict because it gives no key error
from collections import defaultdict 
#we need t sort this value
inv_indx = defaultdict(set)
for idx, text in enumerate(docs_list):
    for word in text:
        id_word = vocabulary[word]
        inv_indx[id_word].add(idx)

In [10]:
# to have a list as the value inside the dict
for key in inv_indx:
    value = inv_indx[key]
    inv_indx[key] = list(value)

In [11]:
# save inv_indx to the .txt file
inv_file = open(r'C:\Users\Daniele\inverted_indx.txt', 'w', encoding = "utf8")
for id_term in inv_indx:
    docks = inv_indx[id_term]
    d = '\t'.join(map(str, docks))
    inv_file.write('{0}\t{1}\n'.format(id_term, d))

In [12]:
query = str(input())

a beautiful house with garden and beach


In [13]:
def SearchEngine(query, vocab, inv_indx):
    doc_list = []
    li = preprocess(query)
    pos_list = []
    for item in li:
        if item not in vocabulary:
            print('No documents found')
            break
        pos_id = vocabulary[item]
        pos_list.append(pos_id)
    result_list = []
    for term_id in pos_list:
        result_list.append(inv_indx[term_id])
    return result_list

In [14]:
def findTheSomething(res_list):
    test = {k:1 for k in res_list[0]}
    for sublist_ind in range(1, len(res_list)):
        for k in res_list[sublist_ind]:
            try:
                test[k] += 1
            except:
                test[k] = 1
    return [doc_id for doc_id in test if test[doc_id]==3]

In [15]:
pos = SearchEngine(query, vocabulary, inv_indx)


In [16]:
doc = findTheSomething(pos)
doc

[1, 35, 207]

In [26]:
g=[]
for i in doc:
    g.append(pd.read_csv(r'C:\Users\Daniele\fileAIR\doc_'+str(i)+'.tsv',sep='\t'))
g1=[[j for j in i] for i in g]
cols = ['1', '2', 'City', '4','Description','6','7','Title','Url']
df1 = pd.DataFrame(g1,columns=cols)[['Title','Description','City','Url']]
df1

,Title,Description,City,Url
0,Unique Location! Alamo Heights - Designer Insp...,"Stylish, fully remodeled home in upscale NW – ...",San Antonio,https://www.airbnb.com/rooms/17481455?location...
1,Monroe Manor,Come relax on the water in our beautiful beach...,Houston,https://www.airbnb.com/rooms/19273801?location...
2,MarBella House Near the Galveston Beach,Location Location Location - Large 3 bdrm hous...,Galveston,https://www.airbnb.com/rooms/7459305?location=...
